# Prep for meeting using Exa.ai search https://exa.ai/

In [31]:
!pip -q install crewai duckduckgo-search
!pip -q install  decouple langchain-exa exa_py==1.0.7 langchain-openai

# install crewai[tools] with "pip install 'crewai[tools]'"

# crewai==0.22.5


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!py -3.11 -m pip install --upgrade pip

In [3]:
!pip show crewai

Name: crewai
Version: 0.22.5
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: D:\repos\ai-agents-crewai\.venv\Lib\site-packages
Requires: click, instructor, langchain, langchain-openai, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [4]:
import os
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY')


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


True

## Tool

In [5]:
from exa_py import Exa
from langchain.agents import tool

class ExaSearchTool:
	@tool
	def search(query: str):
		"""Search for a webpage based on the query."""
		return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

	@tool
	def find_similar(url: str):
		"""Search for webpages similar to a given URL.
		The url passed in should be a URL returned from `search`.
		"""
		return ExaSearchTool._exa().find_similar(url, num_results=3)

	@tool
	def get_contents(ids: str):
		"""Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
		ids = eval(ids)
		contents = str(ExaSearchTool._exa().get_contents(ids))
		print(contents)
		contents = contents.split("URL:")
		contents = [content[:1000] for content in contents]
		return "\n\n".join(contents)

	def tools():
		return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

	def _exa():
		return Exa(api_key=os.environ["EXA_API_KEY"])

## Agents

In [32]:
from textwrap import dedent
from crewai import Agent
from langchain_openai import ChatOpenAI


class MeetingPreparationAgents():
	def research_agent(self):
		return Agent(
			role='Research Specialist',
			goal='Conduct thorough research on people and companies involved in the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Research Specialist, your mission is to uncover detailed information
					about the individuals and entities participating in the meeting. Your insights
					will lay the groundwork for strategic meeting preparation."""),
			llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2),
			max_rpm=500,
			verbose=True
		)

	def industry_analysis_agent(self):
		return Agent(
			role='Industry Analyst',
			goal='Analyze the current industry trends, challenges, and opportunities',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As an Industry Analyst, your analysis will identify key trends,
					challenges facing the industry, and potential opportunities that
					could be leveraged during the meeting for strategic advantage."""),
			llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5),
			max_rpm=500,
			verbose=True
		)

	def meeting_strategy_agent(self):
		return Agent(
			role='Meeting Strategy Advisor',
			goal='Develop talking points, questions, and strategic angles for the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Strategy Advisor, your expertise will guide the development of
					talking points, insightful questions, and strategic angles
					to ensure the meeting's objectives are achieved."""),
			llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5),
			max_rpm=500,
			verbose=True
		)

	def summary_and_briefing_agent(self):
		return Agent(
			role='Briefing Coordinator',
			goal='Compile all gathered information into a concise, informative briefing document',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As the Briefing Coordinator, your role is to consolidate the research,
					analysis, and strategic insights."""),
			llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5),
			max_rpm=500,
			verbose=True
		)

## Tasks

In [33]:
from crewai import Task

class MeetingPreparationTasks():
	def research_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
			async_execution=True,
			agent=agent
		)

	def industry_analysis_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
		)

	def meeting_strategy_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Develop strategic talking points, questions, and discussion angles
				for the meeting based on the research and industry analysis conducted

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
		)

	def summary_and_briefing_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent
		)

## Crew

In [34]:
from crewai import Crew


tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

print("## Welcome to the Meeting Prep Crew")
print('-------------------------------')
participants = input("What are the emails for the participants (other than you) in the meeting?\n")
# andrew-ng@ai-agentsy.com, andrej-karpathy@ai-agentsy.com, demis-hassabis@ai-agentsy.com
context = input("What is the context of the meeting?\n") # Creating a beats alternative
# Comptetitive analysis on AI Agents topic
objective = input("What is your objective for this meeting?\n") # convince him to partner with us
# Convince participants that the company should purchase Agents of Tomorrow, which is a startup that created a promising framework for orchestrating AI Agents 


## Welcome to the Meeting Prep Crew
-------------------------------


In [35]:
print(participants)
print(context)
print(objective)

andrew-ng@ai-agentsy.com, andrej-karpathy@ai-agentsy.com, demis-hassabis@ai-agentsy.com
Comptetitive analysis on AI Agents topic
Convince participants that the company should purchase Agents of Tomorrow, which is a startup that created a promising framework for orchestrating AI Agents 


In [36]:
# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [37]:


# Create Crew responsible for Copy
crew = Crew(
	agents=[
		researcher_agent,
		industry_analyst_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)



2024-04-02 03:26:41,380 - 25996 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [38]:
result = crew.kickoff()


# Print results
print("\n\n################################################")
print("## Here is the result")
print("################################################\n")
print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I need to start by gathering information on each participant and company involved in the meeting. I should focus on recent news, achievements, professional background, and relevant business activities.

Action: search
Action Input: {"query": "Andrew Ng AI Agentsy"}I need to gather information on the current industry trends, challenges, and opportunities related to AI Agents for the competitive analysis meeting. I should search for market reports, recent developments, and expert opinions to provide a comprehensive overview.

Action: search
Action Input: {"query": "AI Agents industry trends 2021"} 

Title: Home - Spring Board
URL: https://springboard.ai/
ID: KS1Gjwu2ZAn-WoFW2_LvTA
Score: 0.17630144953727722
Published Date: 2022-05-27
Author: None
Text: None
Highlights: None
Highlight Scores: None


Title: 首页 | AutoAgents
URL: https://autoagents.ai/
ID: qfF5TW7E0M3HYE9AaQ6MFg
Score: 0.17476804554462433

In [39]:
from IPython.display import display, Markdown
display(Markdown(result))


**Briefing Document**

---

**Participant Bios:**

1. **Name:** [Participant 1]
   - **Role:** CEO
   - **Background:** Extensive experience in technology acquisitions and strategic investments.
   
2. **Name:** [Participant 2]
   - **Role:** CTO
   - **Background:** Technical expert in AI and machine learning applications.

3. **Name:** [Participant 3]
   - **Role:** CFO
   - **Background:** Financial strategist with a focus on ROI and growth opportunities.

---

**Industry Overview:**

The AI Agents sector is experiencing rapid growth and innovation, with key trends and insights from industry reports in 2021 highlighting the importance of AI technologies in various applications. The competitive landscape analysis reveals the strengths and weaknesses of key players, positioning Agents of Tomorrow as a promising contender.

---

**Talking Points:**

1. **Competitive Landscape Analysis:**
   - Overview of key players in the AI Agents sector.
   - Strengths and weaknesses of competitors.
   
2. **Agents of Tomorrow Framework:**
   - Detailed analysis of the innovative framework.
   - Unique features and advantages of Agents of Tomorrow.

3. **Market Trends:**
   - Insights from AI startup founders and analysts.
   - Current industry trends in the AI Agents sector.

4. **Value Proposition:**
   - Articulation of the value of acquiring Agents of Tomorrow.
   - Emphasis on innovation and growth potential.

5. **Implementation Strategy:**
   - Steps required to integrate Agents of Tomorrow.
   - Ensuring a smooth transition into existing systems.

---

**Strategic Recommendations:**

1. **Differentiation:**
   - Highlight how Agents of Tomorrow sets itself apart from competitors.
   
2. **Challenges Addressed:**
   - Specific organizational challenges that Agents of Tomorrow can solve.
   
3. **Scalability and Limitations:**
   - Assessment of scalability and potential limitations of the framework.
   
4. **Success Factors:**
   - Key factors for effective implementation of Agents of Tomorrow.
   
5. **Pricing Model and ROI:**
   - Comparison of pricing with competitors and expected ROI.

---

**Discussion Angles:**

1. **SWOT Analysis:**
   - Comparison of Agents of Tomorrow with competitors.
   - Identification of strategic opportunities.

2. **Use Case Scenarios:**
   - Real-world examples of how Agents of Tomorrow can drive business outcomes.

3. **Stakeholder Alignment:**
   - Importance of aligning key stakeholders for successful adoption.

4. **Risk Mitigation:**
   - Addressing potential risks and proposing mitigation strategies.

5. **Long-Term Vision:**
   - Leveraging Agents of Tomorrow for future success in the AI Agents sector.

---

By focusing on these key areas, the meeting participants will be equipped with the necessary information and strategies to make an informed decision on acquiring Agents of Tomorrow for the organization's growth and innovation in the AI Agents sector.